# [Advent of Code 2020 Day 15](https://adventofcode.com/2020/day/15)

Now this looks like one of those fun logic game questions you see on LeetCode solved either using DP or some insane deductive math.

## Initial setup

In [1]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Input Parsing
The inputs are small enough to send directly.

## Part 1
Lorem ipsum

In [2]:
def part_one(data: str, turns: int = 2020) -> int | str:

    assert turns > 0

    nums: list[int] = intsep(data, ",")
    record: dict[int, list] = defaultdict(list)
    speech: list[int] = []

    for idx, num in enumerate(nums):
        speech.append(num)
        record[num].append(idx + 1)

    if turns <= len(nums):
        return nums[turns - 1]

    for i in range(len(nums) + 1, turns + 5):
        if len(hit_list := (record.get(last_num_spoken := speech[-1]))) == 1:
            #print(f"f({i}) => {last_num_spoken}, -1, -1 -> 0")
            record[0].append(i)
            speech.append(0)
        else:
            to_speak = hit_list[-1] - hit_list[-2]
            #print(f"f({i}) => {last_num_spoken}, {hit_list[-1]}, {hit_list[-2]} -> {to_speak}")
            record[to_speak].append(i)
            speech.append(to_speak)

    return speech[turns - 1]

In [3]:
%%ipytest
def test_part_one_step_by_step():
    assert part_one("0,3,6", turns=1) == 0
    assert part_one("0,3,6", turns=2) == 3
    assert part_one("0,3,6", turns=3) == 6
    assert part_one("0,3,6", turns=4) == 0
    assert part_one("0,3,6", turns=5) == 3
    assert part_one("0,3,6", turns=6) == 3
    assert part_one("0,3,6", turns=7) == 1
    assert part_one("0,3,6", turns=8) == 0
    assert part_one("0,3,6", turns=9) == 4
    assert part_one("0,3,6", turns=10) == 0

def test_part_one_examples_2020():
    assert part_one("0,3,6") == 436
    assert part_one("1,3,2") == 1
    assert part_one("2,1,3") == 10
    assert part_one("1,2,3") == 27
    assert part_one("2,3,1") == 78
    assert part_one("3,2,1") == 438
    assert part_one("3,1,2") == 1836

def test_part_one():
    assert part_one("1,20,8,12,0,14") == 492

...                                                                                          [100%]
3 passed in 0.01s


## Part 2
LUL I fucking knew it, I knew they'd do this, fuck. Time to bring out the ol' math hat.

What can a recurrence relation be for this question? It'd obviously have to start off top-down. Obviously the base case would be if you ask for a turn number for when the game hasn't started:
```
function f(turn_number, starting_numbers) is:
    if turn_number doesn't exceed the starting numbers:
        return the starting number corresponding to the turn_number
    else:
        ?????
```
Dynamic programming only really shines when we have:
- Optimal substructure, and
- Overlapping sub-problems

what could we reify here? Let's trace back a bit and look at the starting example, `[0, 3, 6]`:
1. This is a starting index, so we say 0
2. This is a starting index, so we say 3
3. This is a starting index, so we say 6
4. Take the last number, 6. It's the first time we've stated it, so we say 0.
5. Take the last number, 0. We've stated it twice, on turns 4 and 1, so we say 4 - 1 = 3
6. Take the last number, 3. We've stated it twice, on turns 5 and 2, so we say 5 - 2 = 3
7. Take the last number, 3. We've stated it twice, on turns 6 and 5, so we say 6 - 5 = 1
8. Take the last number, 1. It's the first time we've stated it, so we say 0.
9. Take the last number, 0. We've stated it twice, on turns 8 and 4, so we say 8 - 4 = 4.
10. Take the last number, 4. It's the first time we've stated it, so we say 0.

$f(n)$ will rely on the value of $f(n - 1)$, which then relies on $f(n - 2)$. Well no shit... lmao. But this isn't getting anywhere. What other logical deductions can we make?
- Every new number is immediately proceeded by a 0
  - This means if there are $s$ starting numbers, $f(s+1)=0$
  - ...

This isn't really getting anywhere. Usually how I like to cheese DP questions is I think of as few discriminating states as possible and try to parametrize those. We can find overlapping sub-problems from that point on; we can see in the examples on the page, and in my examples, and in my outputs, that numbers do repeat. So maybe we can capture that?

Looking at turns 4+ in the example above, we see the following inputs:
- Last number,
- Last number's most recent occurrence, and
- Last number's second most recent occurrence

and if we represent non-existence as whitespace, then we can then look at the inputs starting at 4 as:
```
 call          ( prv  |  t1  |  t2  )    =>    ans
f( 4)    =>    (   6  |      |      )    =>     0
f( 5)    =>    (   0  |   4  |   1  )    =>     3
f( 6)    =>    (   3  |   5  |   2  )    =>     3
f( 7)    =>    (   3  |   6  |   5  )    =>     1
f( 8)    =>    (   1  |      |      )    =>     0
f( 9)    =>    (   0  |   8  |   4  )    =>     4
f(10)    =>    (   4  |      |      )    =>     0
f(11)    =>    (   0  |  10  |   8  )    =>     2
f(12)    =>    (   2  |      |      )    =>     0
f(13)    =>    (   0  |  12  |  10  )    =>     2
f(14)    =>    (   2  |  13  |  11  )    =>     2
f(15)    =>    (   2  |  14  |  13  )    =>     1
f(16)    =>    (   1  |  15  |   7  )    =>     8
f(17)    =>    (   8  |      |      )    =>     0
f(18)    =>    (   0  |  17  |  12  )    =>     5
f(19)    =>    (   5  |      |      )    =>     0
f(20)    =>    (   0  |  19  |  17  )    =>     2
```

we notice that:
- the two indices for $f(n)$ are always smaller than $n$
- the second index is always smaller than the first one
- in other words, for $f(n)=$, $n \lt t_1 \lt t_2$

...

OK, I took a half-day break, but now I'm back. I'm going to continue staring at it until it speaks to me (or I die). This one's for `[2, 1, 3]`.

```
f(4) => 3, -1, -1 -> 0
f(5) => 0, -1, -1 -> 0
f(6) => 0, 5, 4 -> 1
f(7) => 1, 6, 2 -> 4
f(8) => 4, -1, -1 -> 0
f(9) => 0, 8, 5 -> 3
f(10) => 3, 9, 3 -> 6
f(11) => 6, -1, -1 -> 0
f(12) => 0, 11, 8 -> 3
f(13) => 3, 12, 9 -> 3
f(14) => 3, 13, 12 -> 1
f(15) => 1, 14, 6 -> 8
f(16) => 8, -1, -1 -> 0
f(17) => 0, 16, 11 -> 5
f(18) => 5, -1, -1 -> 0
f(19) => 0, 18, 16 -> 2
f(20) => 2, 19, 1 -> 18
f(21) => 18, -1, -1 -> 0
f(22) => 0, 21, 18 -> 3
f(23) => 3, 22, 13 -> 9
f(24) => 9, -1, -1 -> 0
f(25) => 0, 24, 21 -> 3
f(26) => 3, 25, 22 -> 3
f(27) => 3, 26, 25 -> 1
f(28) => 1, 27, 14 -> 13
f(29) => 13, -1, -1 -> 0
f(30) => 0, 29, 24 -> 5
f(31) => 5, 30, 17 -> 13
f(32) => 13, 31, 28 -> 3
f(33) => 3, 32, 26 -> 6
f(34) => 6, 33, 10 -> 23
f(35) => 23, -1, -1 -> 0
f(36) => 0, 35, 29 -> 6
f(37) => 6, 36, 33 -> 3
f(38) => 3, 37, 32 -> 5
f(39) => 5, 38, 30 -> 8
f(40) => 8, 39, 15 -> 24
f(41) => 24, -1, -1 -> 0
f(42) => 0, 41, 35 -> 6
f(43) => 6, 42, 36 -> 6
f(44) => 6, 43, 42 -> 1
f(45) => 1, 44, 27 -> 17
f(46) => 17, -1, -1 -> 0
f(47) => 0, 46, 41 -> 5
f(48) => 5, 47, 38 -> 9
f(49) => 9, 48, 23 -> 25
f(50) => 25, -1, -1 -> 0
f(51) => 0, 50, 46 -> 4
f(52) => 4, 51, 7 -> 44
f(53) => 44, -1, -1 -> 0
f(54) => 0, 53, 50 -> 3
```

agh fuck it, I'm just gonna wait it out

In [4]:
def part_two(data: str, turns: int = 30000000) -> int | str:

    assert turns > 0

    nums: list[int] = intsep(data, ",")
    record: dict[int, list] = defaultdict(list)
    speech: list[int] = [0] * (turns + len(nums) + 1)

    for idx, num in enumerate(nums):
        speech[idx] = num
        record[num].append(idx + 1)

    if turns <= len(nums):
        return nums[turns - 1]

    for i in range(len(nums) + 1, turns + 5):
        if len(hit_list := (record.get(last_num_spoken := speech[i - 2]))) == 1:
            record[0].append(i)
            speech[i - 1] = 0
        else:
            to_speak = hit_list[-1] - hit_list[-2]
            record[to_speak].append(i)
            speech[i - 1] = to_speak

    return speech[turns - 1]

In [5]:
%%ipytest
def test_part_two():
    assert part_two("1,20,8,12,0,14") == 63644

.                                                                                            [100%]
1 passed in 21.02s


that...took way less time than I anticipated...

Looking on Reddit, it appears I trolled myself for several hours today... there is no efficient big-brained, mathy, combinatorial bottom-up tabulated DP-optimized solution...

>>> The sequence appears to be the Van Eck sequence (see this fantastic Numberphile video on it). To my surprise, there doesn't seem to be an efficient algorithm for generating the sequence. The sequence does not repeat, nor does it have a known pattern that would help with generating. Many people (a lot smarter than I am) have taken up the challenge to find a more efficient method, but without result. This means I don't have to give it a try. Instead, I have to be careful to use the correct logic and data structures to keep my sequence generator as fast as possible.

[Fuck me man](https://timvisee.com/blog/solving-aoc-2020-in-under-a-second/)